In [3]:
from typing import Any, Dict, List, Optional

from pydantic import BaseModel, root_validator

# Code cobbled together from various sources:
# https://github.com/samuelcolvin/pydantic/discussions/3091
# https://github.com/samuelcolvin/pydantic/issues/2177
# https://github.com/samuelcolvin/pydantic/discussions/3906

class InheritanceAware(BaseModel):

    t: str
    
    # used to register automatically all the submodels in `_types`.
    _subtypes_: Dict[str, type] = {}
    def __init_subclass__(cls):
        cls._subtypes_[cls.__name__] = cls
    
    @classmethod
    def __get_validators__(cls):
        yield cls._convert_to_real_type_

    @classmethod
    def _convert_to_real_type_(cls, data):
        if issubclass(type(data), cls): return data
        data_type = data.get("t")

        if data_type is None:
            raise ValueError("Missing 'type' attribute")

        sub = cls._subtypes_.get(data_type)

        if sub is None:
            raise TypeError(f"Unsupported sub-type: {data_type}")

        return sub(**data)
    
    @classmethod
    def parse_obj(cls, obj):
        return cls._convert_to_real_type_(obj)
    
    @root_validator(pre=True)
    def set_t(cls, values):
        values['t'] = cls.__name__
        return values



In [13]:

class Pet(InheritanceAware):
    name: str

class Cat(Pet):
    age: int

class Dog(Pet):
    name: str
    food: str

class Person(BaseModel):
    pets: List[Pet]

cat_obj = {'t': 'Cat', 'name': 'Garfield', 'age': 7}
dog_obj = {'t': 'Dog', 'name': 'Snoopy', 'food': 'sandwich'}

# Correctly instantiates subclasses:
print(Person(pets=[cat_obj, dog_obj]))


# This doesn't instantiate the subclass because it calls the constructor directly:
cat = Pet(**cat_obj)
print("Using pet constructor:", type(cat))

# Instead use parse_obj():
cat = Pet.parse_obj(cat_obj)
print("Using Pet.parse_obj:", type(cat))

# parse_raw() also works because it calls parse_obj internally:
cat = Pet.parse_raw('{"t": "Cat", "name": "Garfield", "age": 7}')
print("Using Pet.parse_raw:", type(cat))

# Correctly serializes type information as "t":
print("cat.json():", cat.json())

# For this to work, we needed to add the following to _convert_to_real_type_:
#   if issubclass(type(data), cls): return data
print("Instantiate with objects:", Person(pets = [cat]))


pets=[Cat(t='Cat', name='Garfield', age=7), Dog(t='Dog', name='Snoopy', food='sandwich')]
Using pet constructor: <class '__main__.Pet'>
Using Pet.parse_obj: <class '__main__.Cat'>
Using Pet.parse_raw: <class '__main__.Cat'>
cat.json(): {"t": "Cat", "name": "Garfield", "age": 7}
Instantiate with objects: pets=[Cat(t='Cat', name='Garfield', age=7)]
